### Deep Learning in the Eye Tracking World 
#### the tutorial presented during ETRA 2021 (https://etra.acm.org/2021/acceptedtutorials.html)
#### the code downloaded from: https://github.com/kasprowski/etra2021
@author: pawel@kasprowski.pl


# Model transforming a number into a sequence of digits
- input: number in range(0,DATASE_SIZE)
- output: three digits

Examples: 
- input: 234, output: [2,3,4]
- input: 4, output: [0,0,4]


In [2]:
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import RNN, LSTM, RepeatVector
import numpy as np

## Model

In [3]:
model = Sequential()
model.add(Dense(16, input_dim=1) ) 
model.add(RepeatVector(3)) #sequence length
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(128, return_sequences=True))
model.add(Dense(10,activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', optimizer="adam",metrics=['accuracy','mae'])
num_epochs = 0

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 16)                32        
_________________________________________________________________
repeat_vector (RepeatVector) (None, 3, 16)             0         
_________________________________________________________________
lstm (LSTM)                  (None, 3, 128)            74240     
_________________________________________________________________
lstm_1 (LSTM)                (None, 3, 128)            131584    
_________________________________________________________________
lstm_2 (LSTM)                (None, 3, 128)            131584    
_________________________________________________________________
dense_1 (Dense)              (None, 3, 10)             1290      
Total params: 338,730
Trainable params: 338,730
Non-trainable params: 0
__________________________________________________

## Dataset creation

In [4]:
DATASET_SIZE=200

samples = []
labels = []

for i in range(DATASET_SIZE):
    samples.append(i)
    labels.append( list(str(i).rjust(3,'0')) )

samples = np.array(samples)
labels = np.array(labels,dtype=int)

print("Sample (input):",samples[123])
print("Label",labels[123])

# encode labels to one-hot
nlabels = np.zeros((DATASET_SIZE,3,10))
for i in range(DATASET_SIZE):
    for j in range(3):
        x = labels[i][j]
        nlabels[i][j][int(x)] = 1
print("Label encoded (output):\n",nlabels[123])
labels = nlabels

Sample (input): 123
Label [1 2 3]
Label encoded (output):
 [[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]]


In [5]:
TRAINING_SIZE = .5
from sklearn.model_selection import train_test_split
(trainSamples, testSamples, trainLabels, testLabels) = train_test_split(samples, labels,train_size=TRAINING_SIZE)
print('Training samples:',len(trainSamples),' test samples',len(testSamples))

Training samples: 100  test samples 100


In [7]:
EPOCHS=100
BATCH_SIZE = int(len(trainSamples)/4)
print('Training with',len(trainSamples),'samples',EPOCHS,'epochs and batch_size=',BATCH_SIZE)
for x in range(10):
    H = model.fit(trainSamples, trainLabels, epochs=EPOCHS,verbose=0,batch_size=BATCH_SIZE)
    num_epochs += EPOCHS
    print()
    print("Final epochs: {} - accuracy={:6.3f} , loss={:6.3f}, loss improvement={:6.3f}".
          format(num_epochs,H.history['accuracy'][-1] ,H.history['loss'][-1], H.history['loss'][0]-H.history['loss'][-1]))
    check_model(1)
print("Done")

Training with 100 samples 100 epochs and batch_size= 25

Final epochs: 100 - accuracy= 0.580 , loss= 1.039, loss improvement= 1.259
Correct numbers 14 / 200 = 0.07
Correct digits 310/600 = 0.52

Final epochs: 200 - accuracy= 0.693 , loss= 0.820, loss improvement= 0.200
Correct numbers 20 / 200 = 0.1
Correct digits 361/600 = 0.60

Final epochs: 300 - accuracy= 0.657 , loss= 0.824, loss improvement=-0.014
Correct numbers 12 / 200 = 0.06
Correct digits 344/600 = 0.57

Final epochs: 400 - accuracy= 0.673 , loss= 0.730, loss improvement= 0.062
Correct numbers 19 / 200 = 0.095
Correct digits 344/600 = 0.57

Final epochs: 500 - accuracy= 0.750 , loss= 0.627, loss improvement= 0.206
Correct numbers 41 / 200 = 0.205
Correct digits 390/600 = 0.65

Final epochs: 600 - accuracy= 0.773 , loss= 0.548, loss improvement= 0.192
Correct numbers 48 / 200 = 0.24
Correct digits 431/600 = 0.72

Final epochs: 700 - accuracy= 0.803 , loss= 0.476, loss improvement= 0.125
Correct numbers 51 / 200 = 0.255
Correc

In [8]:
def check_model(verbose=1,show_training=1):
    pred = model.predict(samples)
    correct=0
    correct_digits=0
    pp = pred.argmax(axis=2)
    ll = labels.argmax(axis=2)
 
    for i in range(len(pred)):
        if(not show_training and i in trainSamples): continue
        p = pp[i]
        l = ll[i]
        if np.min(p==l)==True: correct+=1
        correct_digits+=np.count_nonzero(np.absolute(p-l)==0)
        if(verbose==2):
            train=''
            if i in trainSamples: train='[T]'
            print(l,'>',p,end=' ')
            if np.min(p==l)==True: print('ok ',train)
            else: print('correct = ', np.count_nonzero(np.absolute(p-l)==0),'/3 ',train,sep='') 
            
    if(verbose>0):
        print('Correct numbers',correct,'/',len(samples), '=',correct/len(samples))
        print('Correct digits {}/{} = {:4.2f}'.format(correct_digits,len(samples)*3,correct_digits/(len(samples)*3)))
    return correct/len(samples)
check_model(2)

[0 0 0] > [9 9 9] correct = 0/3 
[0 0 1] > [0 0 3] correct = 2/3 
[0 0 2] > [0 0 3] correct = 2/3 
[0 0 3] > [0 0 3] ok  [T]
[0 0 4] > [0 0 3] correct = 2/3 
[0 0 5] > [0 0 3] correct = 2/3 
[0 0 6] > [0 0 7] correct = 2/3 
[0 0 7] > [0 0 7] ok  [T]
[0 0 8] > [0 0 7] correct = 2/3 
[0 0 9] > [0 1 1] correct = 1/3 
[0 1 0] > [0 1 1] correct = 2/3 
[0 1 1] > [0 1 1] ok  [T]
[0 1 2] > [0 1 1] correct = 2/3 
[0 1 3] > [0 1 1] correct = 2/3 
[0 1 4] > [0 1 5] correct = 2/3 
[0 1 5] > [0 1 5] ok  [T]
[0 1 6] > [0 1 6] ok  [T]
[0 1 7] > [0 1 7] ok  [T]
[0 1 8] > [0 1 7] correct = 2/3 
[0 1 9] > [0 2 0] correct = 1/3 
[0 2 0] > [0 2 0] ok  [T]
[0 2 1] > [0 2 1] ok  [T]
[0 2 2] > [0 2 3] correct = 2/3 
[0 2 3] > [0 2 3] ok  [T]
[0 2 4] > [0 2 3] correct = 2/3 
[0 2 5] > [0 2 7] correct = 2/3 
[0 2 6] > [0 2 7] correct = 2/3 
[0 2 7] > [0 2 7] ok  [T]
[0 2 8] > [0 2 7] correct = 2/3 
[0 2 9] > [0 2 1] correct = 2/3 
[0 3 0] > [0 3 1] correct = 2/3 
[0 3 1] > [0 3 1] ok  [T]
[0 3 2] > [0 3 3] cor

0.315

In [ ]:
model.save('model_135_999.h5')